In [1]:
# Import necessary libraries
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar
from QuantConnect.Research import QuantBook
from datetime import datetime

qb = QuantBook()

# Define the start and end dates for the backtesting period
start_date = datetime(2009, 5, 1)  # Start from May 2009
end_date = datetime.now()          # Up to the current date


# Set the starting capital (if applicable in QuantBook)
starting_capital = 100000  # $100,000

# Set the global resolution of data
data_resolution = Resolution.Daily
# Add the gold futures contract
gold_futures = qb.AddFuture("GC")

# Optionally, you can filter contracts based on expiration or other criteria
# For example, selecting the front-month contract:
gold_futures.SetFilter(lambda x: x.FrontMonth())

# Fetch historical data for analysis
# (Adjust the start and end dates as needed)
history = qb.History(gold_futures.Symbol, start_date, end_date, Resolution.Daily)

print(history.head())



In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Define the N-day and n-day windows
N = 140  # Large window for trend determination
n_values = [20, 30, 40, 50]  # Example list of n-values for optimization

def determine_trend(data, N):
    highest_high = data['high'].rolling(window=N).max().shift(1)
    lowest_low = data['low'].rolling(window=N).min().shift(1)
    
    trend = ['no_trend'] * len(data)
    current_trend = 'no_trend'

    for i in range(N, len(data)):
        if data['high'][i] > highest_high[i]:
            current_trend = 'uptrend'
        elif data['low'][i] < lowest_low[i]:
            current_trend = 'downtrend'
        trend[i] = current_trend

    return trend

def generate_signals(data, trend, n):
    trend_series = pd.Series(trend, index=data.index)

    highest_high_n = data['high'].rolling(window=n).max()
    lowest_low_n = data['low'].rolling(window=n).min()

    # Imprimir valores para depuración
    print(f"Highest high for last {n} days:\n{highest_high_n.tail()}")
    print(f"Lowest low for last {n} days:\n{lowest_low_n.tail()}")
    
    long_signals = (data['high'] > highest_high_n) & (trend_series.shift(1) == 'uptrend')
    short_signals = (data['low'] < lowest_low_n) & (trend_series.shift(1) == 'downtrend')

    # Imprimir señales para depuración
    print(f"Long signals for n={n}:\n{long_signals[long_signals]}")
    print(f"Short signals for n={n}:\n{short_signals[short_signals]}")

    return long_signals, short_signals

# Lugar donde llamamos a la función generate_signals


In [8]:
import matplotlib.pyplot as plt

# Ajustar el DataFrame para utilizar la fecha como índice
# Suponiendo que 'history' es tu DataFrame original

# Resetear el índice para hacer 'time' una columna
history_reset = history.reset_index()

# Establecer 'time' como el nuevo índice
history_reset.set_index('time', inplace=True)

# Ahora, 'history_reset' tiene 'time' como índice único
# Procedemos a aplicar la estrategia en este DataFrame

# Aplicar funciones de tendencia y señales
trend = determine_trend(history_reset, N=140)

signals = {}
for n in n_values:
    long_signals, short_signals = generate_signals(history_reset, trend, n)
    print(f"n={n}: Number of long signals - {long_signals.sum()}, Number of short signals - {short_signals.sum()}")

    signals[n] = {"long": long_signals, "short": short_signals}

# Verificar que las señales están siendo generadas
for n in n_values:
    print(f"Verificando señales para n={n}")
    if n in signals:
        long_signals, short_signals = signals[n]["long"], signals[n]["short"]
        if not long_signals.any() and not short_signals.any():
            print(f"No se generaron señales comerciales para n={n}")
        else:
            print(f"Señales largas (long) para n={n}: {long_signals[long_signals].index.tolist()}")
            print(f"Señales cortas (short) para n={n}: {short_signals[short_signals].index.tolist()}")
    else:
        print(f"No se encontraron señales para n={n}")

# Asegúrate de que la visualización muestre todas las fechas posibles de señales
plt.figure(figsize=(15, 7))
plt.plot(history_reset['close'], label='Gold Price')

# Usar un enfoque de visualización que no dependa de un DataFrame indexado
for n, signal in signals.items():
    long_dates = history_reset.index[signal["long"]]
    short_dates = history_reset.index[signal["short"]]
    plt.scatter(long_dates, history_reset.loc[long_dates, 'close'], marker='^', color='green', label=f'Long Signal n={n}', alpha=0.7, s=100)
    plt.scatter(short_dates, history_reset.loc[short_dates, 'close'], marker='v', color='red', label=f'Short Signal n={n}', alpha=0.7, s=100)

plt.title('Gold Futures Price and Trading Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()